# Tunnel++

***Goal***: To make an improved version of [Location+](https://library.fangraphs.com/pitching/stuff-location-and-pitching-primer/) that incorporates pitch tunneling.
**Author**: Jensen Holm <br>
**Date**: 2024

In [1]:
import statcast_pitches
import polars as pl
from plotnine import *

## Loading Data

Using all pitches thrown in the statcast-era (2015-present) for our initial data.

**Data Dictionary**: [link](https://baseballsavant.mlb.com/csv-docs) <br>
**statcast_pitches package info**: [link](https://github.com/Jensen-holm/statcast-era-pitches) 

In [ ]:
pitches_df: pl.DataFrame = statcast_pitches.load()

(pitches_df
    .sample(3, seed=42))

pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,…,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length
str,datetime[μs],f64,f64,f64,str,i64,i64,str,str,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,f64,str,i64,i64,i64,f64,f64,f64,f64,i64,i64,i64,i64,f64,str,…,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,str,str,i64,f64,f64,f64,f64
"""FF""",2016-06-24 00:00:00,94.9,-2.57,5.54,"""Withrow, Chris""",425766,519437,null,"""called_strike""",null,null,null,null,8,"""James Loney grounds out, first…","""R""","""L""","""R""","""ATL""","""NYM""","""S""",null,null,1,1,2016,-1.09,1.28,-0.14,1.67,null,null,null,2,7.0,"""Top""",…,5.7,447967,519437,452095,518692,607054,611177,430947,425796,542255,455976,54.5,null,null,null,null,null,null,null,58,3,"""4-Seam Fastball""",6,8,8,6,8,6,8,6,"""Infield shift""","""Standard""",219,0.0,-0.025,null,null
"""SL""",2017-04-13 00:00:00,85.7,-2.44,5.77,"""Kuhl, Chad""",643217,641771,null,"""called_strike""",null,null,null,null,13,"""Andrew Benintendi flies out sh…","""R""","""L""","""R""","""BOS""","""PIT""","""S""",null,null,0,0,2017,-0.25,0.34,-1.03,1.91,null,null,null,1,6.0,"""Bot""",…,6.0,490143,641771,455755,605137,594838,543281,474568,570256,516782,457705,54.51,null,null,null,null,null,null,null,45,1,"""Slider""",1,3,1,3,3,1,1,3,"""Standard""","""Standard""",230,0.0,-0.026,null,null
"""FF""",2022-06-26 00:00:00,96.6,-1.14,5.95,"""Knebel, Corey""",595751,608349,null,"""foul""",null,null,null,null,2,"""Jorge Alfaro strikes out swing…","""R""","""R""","""R""","""SD""","""PHI""","""S""",null,null,3,2,2022,-0.28,1.3,-0.05,3.28,null,null,null,2,7.0,"""Bot""",…,6.4,662254,608349,596117,656555,681082,664761,544369,656941,546318,663837,54.08,null,null,null,null,null,null,null,61,6,"""4-Seam Fastball""",5,6,5,6,6,5,5,6,"""Standard""","""Standard""",196,0.0,0.0,null,null


In [9]:
print(f"Number of pitches: {len(pitches_df):,}")
print(f"Date Range of pitches thrown: {pitches_df["game_date"].min().date()} - {pitches_df["game_date"].max().date()}")
print(f"Columns & Schema: {pitches_df.collect_schema()}")

Number of pitches: 7,011,277
Date Range of pitches thrown: 2015-04-05 - 2024-10-29
Columns & Schema: Schema({'pitch_type': String, 'game_date': Datetime(time_unit='us', time_zone=None), 'release_speed': Float64, 'release_pos_x': Float64, 'release_pos_z': Float64, 'player_name': String, 'batter': Int64, 'pitcher': Int64, 'events': String, 'description': String, 'spin_dir': Int64, 'spin_rate_deprecated': Int64, 'break_angle_deprecated': Int64, 'break_length_deprecated': Int64, 'zone': Int64, 'des': String, 'game_type': String, 'stand': String, 'p_throws': String, 'home_team': String, 'away_team': String, 'type': String, 'hit_location': Float64, 'bb_type': String, 'balls': Int64, 'strikes': Int64, 'game_year': Int64, 'pfx_x': Float64, 'pfx_z': Float64, 'plate_x': Float64, 'plate_z': Float64, 'on_3b': Int64, 'on_2b': Int64, 'on_1b': Int64, 'outs_when_up': Int64, 'inning': Float64, 'inning_topbot': String, 'hc_x': Float64, 'hc_y': Float64, 'tfs_deprecated': Int64, 'tfs_zulu_deprecated': Int

## EDA